In [37]:
import pandas as pd
from IPython.display import display, HTML

raw_train = pd.read_csv("train.csv")
raw_test = pd.read_csv("test.csv")

print("Train set:")
display(raw_train.head())

print("Test set:")
display(raw_test.head())

Train set:


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


Test set:


,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [2]:
X_train, y_train = raw_train['text'].values, raw_train['author'].values
X_test = raw_test['text']

In [3]:
print("Some tokenized sentences from train:")
print(X_train[:2])

Some tokenized sentences from train:
[ 'This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.'
 'It never once occurred to me that the fumbling might be a mere mistake.']


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train)
X_train = tf_transformer.transform(X_train)
X_test = tf_transformer.transform(X_test)

In [5]:
from sklearn.model_selection import train_test_split

X_learn, X_val, y_learn, y_val = train_test_split(X_train, y_train)

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import numpy as np

gnb =  MultinomialNB()
gnb.fit(X_learn, y_learn)
prediction = gnb.predict(X_val)
print("Accuracy on test set: {}".format(accuracy_score(prediction, y_val)))
proba_predicted = gnb.predict_proba(X_val)


Accuracy on test set: 0.7244126659856996
['EAP' 'HPL' 'MWS']
[[1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]]
[[ 0.81684383  0.07909342  0.10406275]
 [ 0.28635869  0.56928428  0.14435703]
 [ 0.8223847   0.09129295  0.08632235]
 [ 0.70008295  0.1231149   0.17680216]
 [ 0.41650073  0.44398039  0.13951888]]


ValueError: Multioutput target data is not supported with label binarization

In [28]:
# Train on all for submission (no tuning so legit)

gnb =  MultinomialNB()
predict_proba = gnb.fit(X_train, y_train).predict_proba(X_test)
print(gnb.classes_)
print(predict_proba[:10])

['EAP' 'HPL' 'MWS']
[[ 0.34932385  0.10587364  0.54480251]
 [ 0.72842776  0.15982642  0.11174581]
 [ 0.38167646  0.49381591  0.12450763]
 [ 0.54648006  0.32133064  0.1321893 ]
 [ 0.63177289  0.12507139  0.24315572]
 [ 0.58732028  0.26760902  0.14507071]
 [ 0.7238056   0.14204926  0.13414514]
 [ 0.33090627  0.20997183  0.4591219 ]
 [ 0.70731164  0.1838161   0.10887226]
 [ 0.70407542  0.09980573  0.19611885]]


In [43]:
# From submission

submission = pd.DataFrame(predict_proba,columns=['EAP','HPL','MWS'])
submission["id"] = raw_test["id"]
submission = submission[["id","EAP","HPL","MWS"]]
display(submission.head())


,id,EAP,HPL,MWS
0,id02310,0.349324,0.105874,0.544803
1,id24541,0.728428,0.159826,0.111746
2,id00134,0.381676,0.493816,0.124508
3,id27757,0.546480,0.321331,0.132189
4,id04081,0.631773,0.125071,0.243156


In [45]:
submission.to_csv('sub1.csv',index=False)